In [1]:
import sys
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.optimize import minimize
from csv import DictReader

from glicko import run_league, read_csv

See [NathanDeMaria/EndGame/ncaa_fb.R](https://github.com/NathanDeMaria/EndGame/blob/f1b61777e1550575ed4d23d1ee2920ecf1bb01dd/ncaa_fb.R) for the `.csv`s

In [2]:
conferences_path = './ncaaf_conferences.csv'
games_path = './ncaaf.csv'

In [3]:
fbs_conferences = {
    # Power 5
    1, 4, 5, 8, 9,
    # Group of 5
    12, 15, 151, 17, 37, 
    # Indep
    18,
}

conferences = pd.read_csv(conferences_path)
fbs_lookup = {
    row.team: (row.team_conf in fbs_conferences)
    for _, row in conferences.iterrows()
}
def is_fbs(team_name: str) -> bool:
    return fbs_lookup[team_name]

In [4]:
league = read_csv(games_path)

In [5]:
def create_offseason_runner(
    init_variance: float,
    variance_over_time: float,
    fbs_advantage: float,
):
    def run_offseason(league, season: int) -> None:
        for team in league.teams:
            try:
                mean, var = team.rating
                team.update_rating(
                    (mean, var + variance_over_time),
                    season,
                    0
                )
            except ValueError:
                # i.e. this was the first season.
                multiplier = .5 if is_fbs(team.name) else -.5
                team.update_rating((1500 + fbs_advantage * multiplier, init_variance), season, 0)
    return run_offseason

In [6]:
def evaluate(x):
    runner = create_offseason_runner(*x)
    
    # reset hack
    for team in league.teams:
        team._ratings = dict()
        team.games = defaultdict(list)
    
    return run_league(league, runner)[0]

In [7]:
init = [132653.8137034986, 23135.41931163794, 557.4684194903585]

In [8]:
result = minimize(
    evaluate,
    x0=np.array(init),
    options=dict(disp=True),
    bounds=((0, None), (0, None), (0, None))
)

init = list(result.x)

In [9]:
print(init)

[132653.8137034986, 23135.419311637717, 557.4684194901366]


In [10]:
runner = create_offseason_runner(*init)

# reset hack
for team in league.teams:
    team._ratings = dict()
    team.games = defaultdict(list)

discrepancy, teams = run_league(league, runner)

In [11]:
# Top teams
for i, t in enumerate(sorted(teams, key=lambda x: -x.rating[0])):
    games = t.games[2018]
    wins = len([g for g in games if g.score > .5])
    losses = len(games) - wins
    print(i + 1, f'({wins}-{losses})', t.name, t.rating)

1 (8-0) Alabama Crimson Tide (2250.6675885791756, 20593.978676097577)
2 (7-0) Clemson Tigers (2170.0007572141644, 17138.410401876372)
3 (6-1) Georgia Bulldogs (2104.303952237035, 16398.711501296537)
4 (7-1) LSU Tigers (2093.634991690654, 12788.098255253431)
5 (7-1) Michigan Wolverines (2086.0648272968615, 12789.493128613585)
6 (6-1) Oklahoma Sooners (2084.4297597070245, 14495.057339194444)
7 (7-1) Ohio State Buckeyes (2080.4956130556484, 15359.139377774363)
8 (6-2) Washington Huskies (2079.7434532596158, 13644.672941522946)
9 (5-2) Penn State Nittany Lions (2062.5063788828365, 15813.707330385287)
10 (6-1) Iowa Hawkeyes (2062.015596849166, 13583.795811910251)
11 (7-0) Notre Dame Fighting Irish (2062.004073296941, 13765.574374613905)
12 (5-2) Wisconsin Badgers (2041.7254680262376, 17007.865736662883)
13 (5-2) Stanford Cardinal (2019.0937003162699, 13457.690165634502)
14 (5-3) Auburn Tigers (2017.0930328063866, 14337.921620732037)
15 (0-0) North Dakota State Bison (2007.0556906238323, 106